In [ ]:
# Instalar las dependencias necesarias
!pip install gradio scikit-learn joblib

# ========== Entrenamiento del Modelo ==========
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Perceptron
import joblib

# Datos de ejemplo (correos spam y no spam)
emails = [
    "¡Enhorabuena, has ganado un iPhone gratis!",
    "Hola John, ¿cómo estás?",
    "¡Reclama ahora tus vacaciones gratis!",
    "Recordatorio de reunión para la próxima semana",
    "Ha sido seleccionado para una oferta especial",
    "¿Comemos mañana?",
    "Oferta por tiempo limitado, haga clic aquí para reclamar",
    "Su extracto bancario está listo",
]
labels = [1, 0, 1, 0, 1, 0, 1, 0]  # 1 = spam, 0 = not spam

# Vectorización de texto
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(emails)

# Entrenamiento del modelo Perceptron
model = Perceptron()
model.fit(X, labels)

# Guardar el modelo y el vectorizador para reutilización
joblib.dump(model, 'spam_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

print("Modelo y vectorizador guardados!")

# ========== Definición de la Aplicación con Gradio ==========
import gradio as gr

# Cargar el modelo entrenado y el vectorizador
model = joblib.load('spam_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

# Función para detectar si un correo es spam o no
def spam_detector(email_text):
    # Preprocesar el texto del correo
    email_vector = vectorizer.transform([email_text])
    
    # Hacer la predicción
    prediction = model.predict(email_vector)
    
    # Devolver el resultado
    return "spam" if prediction[0] == 1 else "not spam"

# Crear la interfaz de usuario con Gradio
interface = gr.Interface(
    fn=spam_detector,
    inputs=gr.Textbox(lines=10, placeholder="Paste your email content here..."),
    outputs="text",
    title="Spam Detector",
    description="Enter the text of an email to check if it is spam or not."
)

# Lanzar la aplicación
interface.launch()
